In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808332 sha256=e42ec935b73d3a8c54b6823803a7d40144bc7456cbc5cb67789b82f9e992bd37
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
import numpy as np
import random

In [ ]:
inventory_items=pd.read_csv('/content/drive/MyDrive/Capstone_Data/inventory_items.csv')
order_items=pd.read_csv('/content/drive/MyDrive/Capstone_Data/order_items.csv')
orders=pd.read_csv('/content/drive/MyDrive/Capstone_Data/orders.csv')
products=pd.read_csv('/content/drive/MyDrive/Capstone_Data/products.csv')
users=pd.read_csv('/content/drive/MyDrive/Capstone_Data/users.csv', )

In [ ]:
data = pd.merge(users, orders, left_on='id', right_on='user_id',suffixes=('_users','_orders'))
data = pd.merge(data, order_items, left_on='order_id', right_on='order_id',suffixes=('','_ord_items'))
data = pd.merge(data, inventory_items, left_on='inventory_item_id', right_on='id',suffixes=('','_inv_items'))
data = pd.merge(data, products, left_on='product_id', right_on='id',suffixes=('','_prod'))

In [ ]:
data.shape

(181759, 55)

In [ ]:
data = data.drop(columns=['first_name', 'last_name', 'email',
       'state', 'street_address', 'postal_code', 'city', 'country', 'latitude',
       'longitude', 'traffic_source', 'created_at_users', 'order_id',
       'user_id', 'status', 'gender_orders', 'created_at_orders',
       'returned_at', 'shipped_at', 'delivered_at', 'num_of_item',
       'id_ord_items', 'user_id_ord_items','inventory_item_id',
       'status_ord_items', 'created_at', 'shipped_at_ord_items',
       'delivered_at_ord_items', 'returned_at_ord_items',
       'id_inv_items', 'product_id_inv_items', 'created_at_inv_items',
       'sold_at', 'cost', 'product_category',
       'product_retail_price','product_sku',
       'product_distribution_center_id', 'id_prod', 'cost_prod', 'category',
       'name', 'brand', 'retail_price', 'department', 'sku',
       'distribution_center_id'])

In [ ]:
data.columns

Index(['id', 'age', 'gender_users', 'product_id', 'sale_price', 'product_name',
       'product_brand', 'product_department'],
      dtype='object')

In [ ]:
data.rename(columns={'id': 'user_id', 'gender_users': 'gender'}, inplace=True)

In [ ]:
data.columns

Index(['user_id', 'age', 'gender', 'product_id', 'sale_price', 'product_name',
       'product_brand', 'product_department'],
      dtype='object')

In [ ]:
# Prepare the interaction data
interaction_data = data.groupby(['user_id', 'product_id']).size().reset_index(name='interaction_count')

# Initialize and fit the dataset
dataset = Dataset()
dataset.fit((row['user_id'] for index, row in interaction_data.iterrows()),
            (row['product_id'] for index, row in interaction_data.iterrows()))

(interactions, weights) = dataset.build_interactions(((row['user_id'], row['product_id']) for index, row in interaction_data.iterrows()))

# Initialize the LightFM model with the WARP loss function
model = LightFM(loss='warp')

# Train the model
model.fit(interactions, epochs=30, num_threads=2)

In [ ]:
def get_preferred_brand(user_id):
    purchased_products = order_items[order_items['user_id']==user_id]['product_id'].values
    purchased_brands = {}
    for i in purchased_products:
        brand = products[products['id']==i]['brand'].values[0]
        if brand in purchased_brands:
            purchased_brands[brand] += 1
        else:
            purchased_brands[brand] = 1
    max_value = max(purchased_brands.values())
    max_keys = [key for key, value in purchased_brands.items() if value == max_value]
    return random.choice(max_keys)

def get_user_age_and_spending(user_id):
    personal_records = data[data['user_id']==user_id]
    age = personal_records['age'].values[0]
    gender = personal_records['gender'].values[0]
    avg_spending = personal_records['sale_price'].sum()/personal_records['sale_price'].count()
    return age, gender, avg_spending

def get_returned_products(user_id):
    return order_items[(order_items['user_id']==user_id) & (order_items['status'].isin(['Cancelled', 'Returned']))]['product_id'].values

def get_best_sellers():
    total_sales_per_product = order_items.groupby('product_id').size()
    top_selling_products = total_sales_per_product.sort_values(ascending=False).head(5)
    return top_selling_products.values

def get_new_arrivals():
    inventory_data_sorted = inventory_items.sort_values(by='created_at', ascending=False)
    return inventory_data_sorted.head(5)['product_id'].values.flatten()

def get_product_gender_preference(product_id):
    gender = inventory_items[inventory_items['product_id']==product_id]['product_department'].values[0]
    return 'M' if gender=='Men' else 'F'

def get_product_brand(product_id):
    return inventory_items[inventory_items['product_id']==product_id]['product_brand'].values[0]

def get_purchased_products(user_id):
    pur_prod = data[data.user_id==user_id]
    return pur_prod[['product_id']].values.flatten()

def get_product_price(product_id):
    return data[data['product_id']==product_id]['sale_price'].values[0]


In [ ]:
def sample_recommendation(model, dataset, user_ids, num_recommendations=5, max_preferred_brand=3, spending_range=(0.8, 1.2)):
    n_users, n_items = dataset.interactions_shape()
    recommend = []
    for user_id in user_ids:
        if user_id in users['id'].values.flatten():
            user_index = dataset.mapping()[0][user_id]
            scores = model.predict(user_index, np.arange(n_items))

            # Get additional data for the user
            preferred_brand = get_preferred_brand(user_id)
            user_age, gender, avg_spending = get_user_age_and_spending(user_id)
            returned_products = get_returned_products(user_id)
            purchased_products = get_purchased_products(user_id)

            # Translate scores to product IDs, excluding returned and purchased products
            product_scores = {
                list(dataset.mapping()[2].keys())[list(dataset.mapping()[2].values()).index(i)]: score
                for i, score in enumerate(scores)
                if i not in returned_products and i not in purchased_products
            }

            # Sort products based on scores, descending
            recommended_products = sorted(product_scores, key=product_scores.get, reverse=True)

            # Apply filtering logic including brand preference, gender suitability, and spending range
            brand_recommendations = []
            other_recommendations = []
            for prod in recommended_products:
                product_gender = get_product_gender_preference(prod)
                product_price = get_product_price(prod)

                # Determine if the product price is within the specified spending range
                if (gender == product_gender) and \
                (avg_spending * spending_range[0] <= product_price <= avg_spending * spending_range[1]):
                    if get_product_brand(prod) == preferred_brand and len(brand_recommendations) < max_preferred_brand:
                        brand_recommendations.append(prod)
                    elif len(other_recommendations) < (num_recommendations - max_preferred_brand):
                        other_recommendations.append(prod)
                if len(brand_recommendations) + len(other_recommendations) == num_recommendations:
                    break

            final_recommendations = brand_recommendations + other_recommendations

            print(f"Recommended items for user {user_id}: {final_recommendations}")
            recommend.append(final_recommendations)
        else:
            final_recommendations = get_new_arrivals()
            print(f"Recommended items for user {user_id}: {final_recommendations}")
            recommend.append(final_recommendations)
    return recommend

In [ ]:
sample_user_ids = [83582]#, 37023, 83582, 15553, 7815, 60193, 867574786, 3095873495]
recommend = sample_recommendation(model, dataset, sample_user_ids)

Recommended items for user 83582: [11619, 11519, 11844, 7292, 2787]


In [ ]:
for i, j in zip(sample_user_ids, recommend):
    print('User ID: ',i,'Recommended Product IDs: ',j)
    if i in users['id'].values:
        user_detail = users[users['id']==i]
        print('Customer Name: ',(user_detail['first_name'].values+' '+user_detail['last_name'].values)[0])
        detail = get_user_age_and_spending(i)
        print(f'Age: {detail[0]} Gender: {detail[1]}    Average Spending: {detail[2]}')
        print('Preferred Brand: ',get_preferred_brand(i).title())
        pur_prod = data[data.user_id==i]
        print('Bought Products:')
        display(pur_prod[['product_name', 'product_id','sale_price','product_brand']])
    else:
        print('New Customer: No details')
    print('\nRecommended Products:')
    data_list = []

    # Iterate over each value of k
    for k in j:
        prod = products[products['id']==k]
        # Get the values for this iteration
        data_f = {
            'Name': prod['name'].values[0],
            'Retail Price': prod['retail_price'].values[0].round(2),
            'Brand': get_product_brand(k),
            'Gender Preference': get_product_gender_preference(k)
        }
        # Append the values to the list
        data_list.append(data_f)
        # Convert the list of dictionaries to a DataFrame
    result_df = pd.DataFrame(data_list)
    display(result_df)

User ID:  83582 Recommended Product IDs:  [11619, 11519, 11844, 7292, 2787]
Customer Name:  Kathryn Ortega
Age: 29 Gender: F    Average Spending: 44.752499465830624
Preferred Brand:  Icebreaker
Bought Products:


,product_name,product_id,sale_price,product_brand
113806,Icebreaker Women's Oasis Long Sleeve Crewe Top,2581,109.989998,Icebreaker
128696,Only Hearts Second Skin Half Slip Nude,11619,44.000000,Only Hearts
135543,Jolt Juniors 5 Inch Short,6383,25.000000,Jolt
151085,Indestructable Aluminum Aluma Wallet - RED,14235,0.020000,marshal



Recommended Products:


,Name,Retail Price,Brand,Gender Preference
0,Only Hearts Second Skin Half Slip Nude,44.00,Only Hearts,F
1,Only Hearts Second Skin Half Slip Black,44.00,Only Hearts,F
2,Only Hearts Delicious With Lace V-Neck Cami Jade,52.00,Only Hearts,F
3,Karen Kane Metallic Skirt,49.99,Karen Kane,F
4,adidas Women's Cotton Fleece Pant,40.00,adidas,F


In [ ]:
import pickle

# Assuming `your_model` is your trained model
with open('pre_train.pkl', 'wb') as file:
    pickle.dump(model, file)